### Historical Shape Indicator (HSI),  ETL for Multiple Stock Survey

In [1]:
import pandas as pd
from pandas import DatetimeIndex
import datetime 
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import scipy
from scipy import stats as scs
import statsmodels
from statsmodels import stats
from statsmodels.stats import weightstats
from statsmodels.stats.power import TTestIndPower
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import seaborn as sb
sb.set()
from alpha_vantage.timeseries import TimeSeries
from datetime import datetime, timedelta
np.set_printoptions(precision=2)
%matplotlib inline
import pickle

#### Raw Data

In [2]:
def get_raw(sym='V'):
    '''
    download data and return data dictionary
    '''
    # download historical prices
    ts = TimeSeries(key='enter your access key')
    # Get json object with the intraday data and another with  the call's metadata
    data, meta_data = ts.get_daily_adjusted(sym, outputsize='full')
    return data

#### Format Raw Data

In [3]:
def format_raw(raw_dict):
    '''
    import raw dictionary
    format column names and sort date ascending
    return dataframe
    '''
    # reformat
    data = raw_dict.copy()
    df_raw = pd.DataFrame.from_dict(data).T
    df_raw.reset_index(level=0, inplace=True)
    df_raw = df_raw.rename(index=str, columns={'index':'date',
                                               '1. open': 'open', 
                                               '2. high': 'high',
                                               '3. low': 'low',
                                               '4. close':'close',
                                               '5. adjusted close':'adj_close',
                                               '6. volume':'volume',
                                               '7. dividend amount':'dividend',
                                               '8. split coefficient':'split',
                                              })
    df_raw = df_raw.sort_values(by='date', ascending=True)
    df_raw = df_raw.reset_index(drop=True)
    df_raw.date = pd.to_datetime(df_raw.date)
    
    return df_raw

In [4]:
def scale_adjusted(df_raw):
    '''
    import raw dataframe
    scale open,high,low, close to adjusted close
    return updated dataframe
    '''
    df = df_raw.copy()
    df_scale = pd.DataFrame()
    close = df.close.to_numpy().astype(float)
    adj = df.adj_close.to_numpy().astype(float)
    scale = adj / close
    df_scale['date'] = df['date'].copy()
    df_scale['open']=df.open.to_numpy().astype(float)*scale
    df_scale['high']=df.high.to_numpy().astype(float)*scale
    df_scale['low']=df.low.to_numpy().astype(float)*scale
    df_scale['close']=df.close.to_numpy().astype(float)*scale
    
    return df_scale

#### Preprocess Data

In [5]:
def compute_log_returns(prices):
    '''
    compute log returns
    '''
    return np.log(prices) - np.log(prices.shift(1))

In [6]:
def shift_returns(returns, shift_n):
    '''
    compute shift returns for trade assessment
    '''
    return returns.shift(shift_n)

In [7]:
def compute_proj(prices, lookahead_days):
    '''
    compute projected future lookahead returns
    lookahead_days is the number of days ahead we want to predict
    '''
    return (prices.shift(-lookahead_days) - prices)/prices

In [8]:
def compute_day_shape(prices, sigmas, dayspan):
    '''
    compute one day shape
    '''
    abs_deltas = (prices) - (prices.shift(dayspan))
    s_ratios = abs_deltas / sigmas
    ups = 3*(s_ratios>1)
    downs = 1*(s_ratios<-1)
    neuts = 2*((s_ratios>=-1)&(s_ratios<=1))      
    return (ups+downs+neuts)

In [9]:
def compute_shape(dayshape, dayspan):
    '''
    compute 5 day shape ordinals
    '''
    ago5s = 10000*(dayshape.shift(4*dayspan))
    ago4s = 1000*(dayshape.shift(3*dayspan))
    ago3s = 100*(dayshape.shift(2*dayspan))
    ago2s = 10*(dayshape.shift(1*dayspan))
    return (ago5s+ago4s+ago3s+ago2s+dayshape)

In [10]:
def preprocess(df):
    '''
    compute statistics 
    add return parameters
    add lookahead projections of 7 days
    use day shape spans of 1, 3 and 5 days
    build shape ordinals
    '''
    df_for = df.copy()
    
    # raw data overlaps
    shifts = [['o1','h1','l1','c1'],
              ['o2','h2','l2','c2'],
              ['o3','h3','l3','c3'],
              ['o4','h4','l4','c4'],
             ]
    # format df to calculate price estimates and standard deviations
    for j, shift in zip(range(1,6),shifts):
        df_for[shift[0]] = df_for.open.shift(-j)
        df_for[shift[1]] = df_for.high.shift(-j)
        df_for[shift[2]] = df_for.low.shift(-j)
        df_for[shift[3]] = df_for.close.shift(-j)

    # define price estimate columns for 1,3,5 day spans
    p1_col = df_for.loc[:,"open":"close"].astype(float)
    p3_col = df_for.loc[:,"open":"c2"].astype(float)
    p5_col = df_for.loc[:,"open":"c4"].astype(float)  
    p_cols = [p1_col, p3_col, p5_col]

    # compute price estimates and standard deviations for spans
    stats = [['pe1','sd1'],['pe3','sd3'],['pe5','sd5']]
    for stat, p_col in zip(stats, p_cols):
        df_for[stat[0]] = p_col.mean(axis=1)
        df_for[stat[1]] = p_col.std(axis=1)

    # keep date but leave raw data behind
    df_prep = df_for[['date','pe1','sd1','pe3','sd3','pe5','sd5']].copy()
    
    # add daily returns to df based on 1 day price estimates
    daily_returns = compute_log_returns(df_prep['pe1'])
    df_prep['log_ret'] = daily_returns
    
    # compute shift returns
    shift_1dlog = shift_returns(df_prep['log_ret'],-1)
    df_prep['shift_ret'] = shift_1dlog
    
    # add projections to df
    lookahead_days = 7
    aheads = compute_proj(df_prep['pe1'], lookahead_days)
    df_prep['proj'] = aheads
    
    # add day shapes to df
    dayshapes = ['ds1','ds3','ds5']
    dayspans = [1,3,5]
    for shape, stat, span in zip(dayshapes, stats, dayspans):
        df_prep[shape] = compute_day_shape(df_prep[stat[0]], df_prep[stat[1]], span)
        
    # add shapes to df
    shapes = ['shp1','shp3','shp5']
    for shape, dayshape, span in zip(shapes, dayshapes, dayspans):
        df_prep[shape] = compute_shape(df_prep[dayshape], span)

    #trim the head then format
    df_trim = df_prep[25:].copy()
    df_trim[['shp1','shp3','shp5']] = df_trim[['shp1','shp3','shp5']].astype(int)
    
    return df_trim  

In [11]:
def test_train_split(df_mkt, test_year):
    '''
    split preprocessed data into train and test dataframes
    train data comes from years prior to test year
    data in years beyond the test year is not used
    '''
    df = df_mkt.copy()
    years = df.date.map(lambda x: x.strftime('%Y')).astype(int)
    
    #train = years < test_year for 2 years behind
    train = ((test_year-3 < years) & (years < test_year))
    test = np.isin(years, test_year)

    df_train = df[train].copy()
    df_test = df[test].copy()
    
    return df_train, df_test

#### Shape Ranks

In [12]:
def compute_shaperank(df_train, shapename):
    '''
    enter preprocessed train data and shapename
    return HSI dataframe for that shapename
    '''
    shapes = df_train[shapename]
    projs = df_train['proj']
    
    s_list = list(set(shapes))
    p_avgs = []
    p_stds = []
    for shape in s_list:
        p_avgs.append((projs*(shapes==shape)).mean())
        p_stds.append((projs*(shapes==shape)).std())
    
    # initiate dataframe build
    df_shape = pd.DataFrame()
    df_shape['shape'] = s_list
    df_shape['p_avg'] = p_avgs
    df_shape['p_std'] = p_stds
    
    # shape ratio as a mini sharpe
    df_shape['p_srs'] = df_shape['p_avg']/df_shape['p_std']
    df_shape = df_shape.sort_values(by=['p_srs'])
    df_shape = df_shape.reset_index(drop=True)
    
    # normalize shape ratios into indicator
    short_range = df_shape['p_srs'].max() - df_shape['p_srs'].min() 
    short_min = df_shape['p_srs'].min()
    df_shape['HSI'] = (df_shape['p_srs'] - short_min)/short_range
    
    return df_shape     

In [13]:
def build_hsi(df_train):
    df1 = compute_shaperank(df_train, 'shp1')
    df3 = compute_shaperank(df_train, 'shp3')
    df5 = compute_shaperank(df_train, 'shp5')
    df_hsi = pd.concat({'shp1':df1, 'shp3':df3, 'shp5':df5}, axis=1)
    return df_hsi

In [14]:
def assign_hsi(df, df_shape):
    '''
    for daily market data
    lookup the HSI figures given shape ordinals
    return updated dataframe with daily HSI assignment
    '''
    df_mkt = df.copy()
    
    # HSI lookups
    shapenames = ['shp1','shp3','shp5']
    hsi_names = ['hsi1','hsi3','hsi5']
    
    for sname, hsi_name in zip(shapenames, hsi_names):
        lookups = []
        s_list = df_shape[sname]['shape'].tolist()
        for i,nrows in df_mkt.iterrows():
            shp = nrows[sname]
            # assign 0.5's for unknown shapes
            if shp in s_list:
                lookups.append(np.asscalar(df_shape[sname][df_shape[sname]['shape']==shp]['HSI'].values))
            else:
                lookups.append(0.5)
        df_mkt[hsi_name] = lookups
        
    # compile three into the average of the two closest
    nearest_two = []
    for i,nrows in df_mkt.iterrows():
        v1 = nrows['hsi1']
        v2 = nrows['hsi3']
        v3 = nrows['hsi5']
        diffs = np.abs([v1-v2, v2-v3, v1-v3])
        sums = [v1+v2, v2+v3, v1+v3]
        nearest_two.append(np.max((diffs==np.amin(diffs))*sums)/2)
        
    df_mkt['HSC'] = nearest_two

    return df_mkt 

#### Trade Rules

In [15]:
def compute_trades(indicator, highT, lowT):
    '''
    compare HSC to thresholds
    return binaries of in/out days
    '''
    trades = []
    inout = 0
    for ind in indicator:
        # from out to enter
        if inout == 0:
            if ind > highT:
                trades.append(1)
                inout = 1
            else:
                trades.append(0)
        # from in to exit
        else:
            if ind < lowT:
                trades.append(0)
                inout = 0
            else:
                trades.append(1)
    return trades

In [16]:
def opt_tresh(seedLow, seedHigh, step_range, df):
    '''
    successive approximation applied to optimizing thresholds
    '''
    df_mkt = df.copy()
    bestL = 0
    bestH = 0
    bestR = 0
    for i in range(20):
        t_low = seedLow + step_range*i/20

        for j in range(20):
            t_high = seedHigh + step_range*j/20

            trade = compute_trades(df_mkt['HSC'], t_high, t_low)
            returns = df_mkt['shift_ret']*trade
            expret = (np.exp(returns[1:].T.sum())-1)*100

            if expret > bestR:
                bestL = t_low
                bestH = t_high
                bestR = expret
                
    return bestL, bestH

In [17]:
def thresholds(df_train):
    '''
    determine trade rule thresholds
    '''
    # trim leader NaN's
    df = df_train.iloc[:-7].copy()
    
    low = 0.25
    high = 0.75
    res = 0
    r_values = [0.5,0.25,0.125]
    for r in r_values:
        low, high = opt_tresh((low-(r/2)),(high-(r/2)),r,df)
    
    return low, high

#### Analysis Functions

In [18]:
def compute_trade_returns(df):
    '''
    compute trade returns
    '''
    return df['shift_ret']*df['trade']

In [19]:
def statistical_test(df):
    '''
    Unequal Variance Stats Test of equal Sample Size
    This is a two-sided test for the null hypothesis that:
    2 independent samples have identical average (expected) values.
    With a small p_value, the null hypothesis is rejected
    '''
    all_ins = df[df['trade']==1]['shift_ret'].dropna()
    all_outs = df[df['trade']==0]['shift_ret'].dropna()
    
    if len(all_ins)<len(all_outs):
        all_outs = np.asarray(np.random.choice(all_outs, len(all_ins)))
    else:
        all_ins = np.asarray(np.random.choice(all_ins, len(all_outs)))
    
    results = statsmodels.stats.weightstats.ttest_ind(all_ins, all_outs,
                                                      alternative="two-sided",
                                                      usevar="unequal")
    t_value = results[0]
    p_value = results[1]
    
    return t_value, p_value

In [20]:
def get_expected_return(returns):
    '''
    compute integrated return in percentage
    '''
    return (np.exp(returns[1:].T.sum())-1)*100

In [21]:
def get_volatility(returns):
    '''
    compute annualized volatility
    '''
    return np.std(returns)*np.sqrt(252)   

In [22]:
def get_years(df_mkt):
    '''
    compute years for sharpe
    '''
    df = df_mkt.copy()
    df = df.reset_index(drop=True)
    return np.asscalar((df['date'].tail(1)-df['date'][0])/timedelta(days=365))

In [23]:
def get_sharpe(returns, years, vol_year):
    '''
    compute sharpe ratio assuming 3.5% risk free interest rate
    '''
    ret_year = (np.exp(returns[1:].T.sum())-1)/years
    risk_free = 0.035
    return (ret_year - risk_free) / vol_year

In [24]:
def get_benchmark(df_mkt, exp_return):
    '''
    compute beat the market percentage
    calculates S&P500 returns using same trade days
    converts log returns to simple percentage
    returns difference in percentage returns
    '''
    df_spy = pd.read_csv('spy_index_092419.csv')
    df_spy['date'] = pd.to_datetime(df_spy['date'])
    df_bench = pd.merge(df_spy[['date', 'shift_ret']], df_mkt[['date','trade']], on='date', how='inner')
    bench_returns = df_bench['shift_ret']*df_bench['trade']
    bench_return = (np.exp(bench_returns[1:].T.sum())-1)*100
    beat_percent = exp_return - bench_return
    return beat_percent

#### ETL Pipeline

In [25]:
def run_etl(ticker, equity):
    '''
    run ETL pipeline
    '''
    print('Runnning ETL for '+ ticker)
    dict_raw = get_raw(ticker)
    
    print('formatting')
    df_for = format_raw(dict_raw)
    df_scale = scale_adjusted(df_for)
    
    print('preprocessing')
    df_pre = preprocess(df_scale)
    df_pre['symbol'] = ticker
    
    print('begin test itterations')
    years = df_pre.date.map(lambda x: x.strftime('%Y')).astype(int).unique().tolist()
    df_res = pd.DataFrame() 
    
    for test_year in years[3:]:
        
        print('starting test year {}'.format(test_year))
        results = [ticker, equity, test_year]
        print('test-train split')
        df_train, df_test = test_train_split(df_pre[:-7], test_year)
        est_price = np.asscalar(df_test['pe1'].tail(1).values)
        results.append(est_price)

        print('training shapes')
        df_shape = build_hsi(df_train)
        df_train = assign_hsi(df_train, df_shape)
        df_test = assign_hsi(df_test, df_shape)

        print('optimizing trade thresholds')
        lowT, highT = thresholds(df_train)
        results.append(lowT)
        results.append(highT)
        
        print('computing trades')
        trades = compute_trades(df_test['HSC'], highT, lowT)
        df_test['trade'] = trades
        num_trades = ((np.diff(trades))==-1).sum() + trades[-1]
        results.append(num_trades)

        print('evaluating performance')
        returns = compute_trade_returns(df_test)
        results.append(np.count_nonzero(returns))

        tval, pval = statistical_test(df_test)
        results.append(tval)
        results.append(pval)
        print('t-value, p-value = ', tval, pval)

        exp_ret = get_expected_return(returns)
        results.append(exp_ret)
        print('expected return = ', exp_ret)

        vol = get_volatility(returns)
        results.append(vol)
        print('volatility = ', vol)

        years = get_years(df_test)
        results.append(years)
        print('years = ', years)

        sharpe = get_sharpe(returns, years, vol)
        results.append(sharpe)
        print('sharpe ratio = ', sharpe)

        beat_percent = get_benchmark(df_test, exp_ret)
        results.append(beat_percent)
        print('beat percent = ', beat_percent)

        print('saving result')
        df_res = df_res.append(pd.Series(results),ignore_index=True)
        
    print('formatting summary')
    cols = ['symbol','equity','test_year','price$','lowT','highT','#trades','in_days',
            't-val','p-val','exp_ret%','volatility','years','sharpe','beat%']
    df_res.columns = cols
    df_res.test_year = df_res.test_year.astype(int)
    df_res.in_days = df_res.in_days.astype(int)
       
    return df_res, df_shape

#### Build Stock Symbol Dictionary

In [26]:
# import candidates
df_stocks = pd.read_csv('street_stocks_092419.csv')

# Add symbol ordinals
df_stocks['sym'] = df_stocks['symbol'].astype('category')
df_stocks['sym'].cat.categories = np.arange(len(df_stocks['symbol'].unique()))
df_stocks['sym'] = df_stocks['sym'].astype('float')

# Build Symbol to Category Dictionary
symbols = df_stocks['symbol'].to_numpy()
sym_cats = df_stocks['sym'].to_numpy()
d_symcat = dict(zip(symbols,sym_cats))

# Pickle for use later
with open('dict_symbol_cat.pkl', 'wb') as handle:
    pickle.dump(d_symcat, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Run Stock Survey

In [27]:
# ticker list for etl runs
t_list = df_stocks.symbol.tolist()
e_list = df_stocks.equity.tolist()
df_res = pd.DataFrame()
hiccup = []
# run etl
for ticker, equity in zip(t_list,e_list):
    try:
        df_results, df_shape = run_etl(ticker, equity)
        df_res = pd.concat([df_res,df_results], ignore_index=True)
    except:
        hiccup.append(ticker)

df_res.to_csv('stock_survey_102819.csv', index=None)
df_res.head()

Runnning ETL for ABT
formatting
preprocessing
begin test itterations
starting test year 2002
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  4.051458631785757 7.445158032529195e-05
expected return =  30.835651305562074
volatility =  0.1471484270795104
years =  0.9945205479452055
sharpe ratio =  1.8692381051632048
beat percent =  24.33191651526481
saving result
starting test year 2003
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.7961658785335957 0.42689477562873557
expected return =  4.050213915499512
volatility =  0.12056843966240151
years =  0.9945205479452055
sharpe ratio =  0.04748582176954402
beat percent =  -0.5232958657952258
saving result
starting test year 2004
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.5619687195901117 0.11959919991089454
expected r

starting test year 2012
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.5427284655622722 0.12432177422817405
expected return =  32.555377344065036
volatility =  0.21271717844112364
years =  0.9945205479452055
sharpe ratio =  1.3743481381486984
beat percent =  20.31036569441855
saving result
starting test year 2013
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.2438422003386027 0.21515397094413108
expected return =  20.43074144030188
volatility =  0.13527462021507222
years =  0.9945205479452055
sharpe ratio =  1.2599042955542177
beat percent =  7.574796602507904
saving result
starting test year 2014
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.21509427040335755 0.8300236898100064
expected return =  -14.750325604945324
volatility =  0.14975034562862508
years =  

preprocessing
begin test itterations
starting test year 2002
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.5976725758685588 0.5510746850293435
expected return =  -4.606428226428072
volatility =  0.1794686436230459
years =  0.9945205479452055
sharpe ratio =  -0.45310466676009525
beat percent =  16.19979441822371
saving result
starting test year 2003
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.0852207458130791 0.9321560251253482
expected return =  17.652102320703044
volatility =  0.1019681190015687
years =  0.9945205479452055
sharpe ratio =  1.3974327684304313
beat percent =  11.773174813052467
saving result
starting test year 2004
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.191316685030163 0.23515136741883416
expected return =  52.39811927699625
volatility

sharpe ratio =  -1.246139246869246
beat percent =  -11.496195955351894
saving result
starting test year 2003
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.9808016869225324 0.3277158929109878
expected return =  42.52521503914015
volatility =  0.1254124730387705
years =  0.9945205479452055
sharpe ratio =  3.1304313513100976
beat percent =  31.283442457629793
saving result
starting test year 2004
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.46741902882043107 0.6406814353234946
expected return =  11.522746036519948
volatility =  0.08303969625304318
years =  0.9972602739726028
sharpe ratio =  0.9699459771241037
beat percent =  12.7277773740607
saving result
starting test year 2005
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.10481338452986103 0.9167607248424569

saving result
starting test year 2011
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.0682685507560485 0.2867904420080535
expected return =  14.614238901875742
volatility =  0.12278143826563898
years =  0.989041095890411
sharpe ratio =  0.9183936665109882
beat percent =  14.973784881804065
saving result
starting test year 2012
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.5328123598012455 0.5947808274596289
expected return =  16.415503329881286
volatility =  0.12465995133910511
years =  0.9945205479452055
sharpe ratio =  1.0433139700992846
beat percent =  7.233560931130588
saving result
starting test year 2013
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  3.0823871434400374 0.0023424244283884285
expected return =  53.26558007592175
volatility =  0.095799730739295

...

sharpe ratio =  3.0194706201185086
beat percent =  12.595209424105413
saving result
starting test year 2018
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.8134401535861961 0.07379071589812088
expected return =  15.921801520792545
volatility =  0.12346461426695166
years =  0.9945205479452055
sharpe ratio =  1.0132073079742339
beat percent =  22.809069904492542
saving result
starting test year 2019
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  3.222683447982389 0.0014979090372919718
expected return =  38.42444030737351
volatility =  0.08655212142701106
years =  0.7890410958904109
sharpe ratio =  5.222014274934946
beat percent =  24.077563675584624
saving result
formatting summary
Runnning ETL for BIO.B
Runnning ETL for BX
Runnning ETL for BRC
Runnning ETL for BR
Runnning ETL for CCMP
Runnning ETL for CACI
Runnning ETL for CWT
Runnning ETL 

years =  0.9945205479452055
sharpe ratio =  2.8883085330263514
beat percent =  17.323675407475392
saving result
starting test year 2016
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.6325496492846052 0.10428936223023413
expected return =  32.2356301963318
volatility =  0.1280905265321632
years =  0.989041095890411
sharpe ratio =  2.271269584253521
beat percent =  15.331414529793584
saving result
starting test year 2017
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.3812304720438313 0.16987916284968932
expected return =  37.1148852350647
volatility =  0.0920797551752726
years =  0.9863013698630136
sharpe ratio =  3.7066095242381643
beat percent =  22.22472965978881
saving result
starting test year 2018
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.66675662922764

volatility =  0.15854852850342468
years =  0.989041095890411
sharpe ratio =  0.11154561210306516
beat percent =  6.361969421966984
saving result
starting test year 2006
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.276315714000813 0.02386432696577362
expected return =  41.66611810240042
volatility =  0.15176725001231617
years =  0.9863013698630136
sharpe ratio =  2.5529100767136375
beat percent =  31.02700093619921
saving result
starting test year 2007
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.2249995586651796 0.22221497243082078
expected return =  24.968280949288445
volatility =  0.20564387998732117
years =  0.9917808219178083
sharpe ratio =  1.054016312374852
beat percent =  11.248858622075364
saving result
starting test year 2008
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performanc

expected return =  -26.099668188905
volatility =  0.18630030543415627
years =  0.9917808219178083
sharpe ratio =  -1.600424846977426
beat percent =  -24.47963981817299
saving result
starting test year 2008
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.6923851598501387 0.0929837839374175
expected return =  -16.378710463664582
volatility =  0.3276103159690624
years =  0.9972602739726028
sharpe ratio =  -0.608152611496636
beat percent =  11.821814397162086
saving result
starting test year 2009
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.16121737477349676 0.8720551128359078
expected return =  -1.0906456789739227
volatility =  0.17814784499023234
years =  0.9945205479452055
sharpe ratio =  -0.25802471797106147
beat percent =  1.9006629756538218
saving result
starting test year 2010
test-train split
training shapes
optimizing trade thresh

,symbol,equity,test_year,price$,lowT,highT,#trades,in_days,t-val,p-val,exp_ret%,volatility,years,sharpe,beat%
0,ABT,ABBOTT LABORATORIES,2002,11.314899,0.31250,0.79375,13.0,99,4.051459,0.000074,30.835651,0.147148,0.994521,1.869238,24.331917
1,ABT,ABBOTT LABORATORIES,2003,14.015966,0.27500,0.73750,20.0,100,-0.796166,0.426895,4.050214,0.120568,0.994521,0.047486,-0.523296
2,ABT,ABBOTT LABORATORIES,2004,15.372593,0.41250,0.60000,38.0,128,1.561969,0.119599,15.532756,0.103041,0.997260,1.171907,9.077632
3,ABT,ABBOTT LABORATORIES,2005,13.312641,0.40625,0.65625,18.0,99,1.579806,0.115950,8.191178,0.074593,0.989041,0.641070,5.821230
4,ABT,ABBOTT LABORATORIES,2006,16.874145,0.33750,0.80000,19.0,94,0.275280,0.783407,12.755089,0.082894,0.986301,1.137870,6.819800


#### Data Server Hiccup Retries

In [28]:
print(hiccup)

['BIO.B', 'BX', 'BRC', 'BR', 'CCMP', 'CACI', 'CWT', 'CNI', 'CCBG', 'CTRE', 'CSL', 'CTLT', 'CBOE', 'CE', 'CF', 'GIB', 'CHEF', 'CB', 'CHD', 'CINF', 'CSCO', 'CHCO', 'CME', 'KO', 'CNS', 'CIGI', 'CBU', 'CWCO', 'CACC', 'CSX', 'CW', 'DHI', 'DHR', 'DRI', 'DE', 'DAL', 'DIOD', 'DFS', 'DTV', 'DTE', 'ETN', 'ECL', 'ESLT', 'EME', 'EBF', 'ENTG', 'EPAM', 'ESE', 'EVRG', 'EXPD', 'EXR', 'FSS', 'FNF', 'FAF', 'FCAP', 'FFIN', 'FCFS', 'FLO', 'FMC', 'WILC', 'GRMN', 'GWR', 'GNTX', 'GGG', 'THG', 'HE', 'HEI.A', 'HXL', 'HON', 'ICLR', 'IDA', 'IEX', 'ITW', 'IOR', 'INFY', 'IR', 'ICE', 'JJSF', 'KSU', 'KNSL', 'KLAC', 'LHX', 'LRCX', 'LDOS', 'LIN', 'LMT', 'MGIC', 'MGA', 'MAIN', 'MANT', 'MTZ', 'MDC', 'MEDP', 'MDT', 'MRK', 'MTH', 'MSFT', 'NDAQ', 'NBHC', 'NEU', 'NEE', 'NCBS', 'NSC', 'NVS', 'NVR', 'ODFL', 'OKE', 'OTTR', 'PCAR', 'PEP', 'PRFT', 'POOL', 'TROW', 'PGR', 'PSA', 'PHM', 'ROLL', 'RSG', 'QSR', 'ROST', 'RPM', 'SAFT', 'SAIA', 'JBSS', 'SPNS', 'SHW', 'SLP', 'SITE', 'SPXC', 'SCL', 'SNPS', 'SYY', 'TGT', 'TMHC', 'TDY', 'TER

In [29]:
len(hiccup)

153

In [30]:
# ticker list for etl runs
t_list = df_stocks.symbol.tolist()
e_list = df_stocks.equity.tolist()
hiccup2 = []
# run etl
for ticker, equity in zip(t_list,e_list):
    if ticker in hiccup:
        try:
            df_results, df_shape = run_etl(ticker, equity)
            df_res = pd.concat([df_res,df_results], ignore_index=True)
        except:
            hiccup2.append(ticker)

df_res.to_csv('stock_survey_102819b.csv', index=None)
df_res.head()

Runnning ETL for BIO.B
Runnning ETL for BX
Runnning ETL for BRC
Runnning ETL for BR
Runnning ETL for CCMP
Runnning ETL for CACI
Runnning ETL for CWT
Runnning ETL for CNI
Runnning ETL for CCBG
Runnning ETL for CTRE
Runnning ETL for CSL
Runnning ETL for CTLT
Runnning ETL for CBOE
Runnning ETL for CE
Runnning ETL for CF
Runnning ETL for GIB
Runnning ETL for CHEF
Runnning ETL for CB
Runnning ETL for CHD
Runnning ETL for CINF
Runnning ETL for CSCO
Runnning ETL for CHCO
Runnning ETL for CME
Runnning ETL for KO
Runnning ETL for CNS
Runnning ETL for CIGI
Runnning ETL for CBU
Runnning ETL for CWCO
Runnning ETL for CACC
Runnning ETL for CSX
Runnning ETL for CW
Runnning ETL for DHI
Runnning ETL for DHR
Runnning ETL for DRI
Runnning ETL for DE
Runnning ETL for DAL
Runnning ETL for DIOD
Runnning ETL for DFS
Runnning ETL for DTV
Runnning ETL for DTE
Runnning ETL for ETN
Runnning ETL for ECL
Runnning ETL for ESLT
Runnning ETL for EME
Runnning ETL for EBF
Runnning ETL for ENTG
Runnning ETL for EPAM
Ru

saving result
starting test year 2013
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.8326201950444074 0.406865063766236
expected return =  1.2760475325017895
volatility =  0.1385160730610883
years =  0.9945205479452055
sharpe ratio =  -0.16004798963677955
beat percent =  -21.681751596368294
saving result
starting test year 2014
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  3.1092078891774637 0.002204565664060532
expected return =  21.153000202773175
volatility =  0.12538193289417623
years =  0.9945205479452055
sharpe ratio =  1.41723334839905
beat percent =  0.025001016508596763
saving result
starting test year 2015
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.7627635681460058 0.44658244300403815
expected return =  21.593073861170865
volatility =  0.1134586497

sharpe ratio =  0.0023743736277040115
beat percent =  0.6357388587275636
saving result
starting test year 2006
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.6979261323683785 0.4861896905689982
expected return =  -1.1115083891902633
volatility =  0.1449250680516704
years =  0.9863013698630136
sharpe ratio =  -0.3192647116133933
beat percent =  -5.43275395757915
saving result
starting test year 2007
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.5258990131428849 0.5994288972161521
expected return =  -23.86758085780486
volatility =  0.19391949871232894
years =  0.9917808219178083
sharpe ratio =  -1.421485651121405
beat percent =  -21.604515305058538
saving result
starting test year 2008
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -1.4436871095928436 0.1506273594

volatility =  0.0977850302716671
years =  0.9945205479452055
sharpe ratio =  2.617144739305009
beat percent =  13.012668834959596
saving result
starting test year 2013
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.9976782779134858 0.04710103784396845
expected return =  31.28174996957076
volatility =  0.11276205801179916
years =  0.9945205479452055
sharpe ratio =  2.4790343225331584
beat percent =  11.156181212670923
saving result
starting test year 2014
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.5260674537966937 0.5993250907303094
expected return =  1.7163072442934135
volatility =  0.1030502342289103
years =  0.9945205479452055
sharpe ratio =  -0.17217200239495067
beat percent =  -2.651755787145982
saving result
starting test year 2015
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating perform

years =  0.989041095890411
sharpe ratio =  5.021785313286966
beat percent =  139.2575649444291
saving result
starting test year 2006
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.1143669330697916 0.03557478513403095
expected return =  54.638695251274186
volatility =  0.2719211925988841
years =  0.9863013698630136
sharpe ratio =  1.9085517212778647
beat percent =  44.45348174709449
saving result
starting test year 2007
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.21148297767762753 0.8328246525462109
expected return =  35.236134517701686
volatility =  0.2735246560272363
years =  0.9917808219178083
sharpe ratio =  1.17094185026683
beat percent =  26.167652974497454
saving result
starting test year 2008
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -1.20470081675

...

beat percent =  32.163163487838254
saving result
starting test year 2016
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.27478266785282407 0.7838375545879727
expected return =  20.712700898536962
volatility =  0.09855221132006138
years =  0.989041095890411
sharpe ratio =  1.7698440528092476
beat percent =  10.58184322354554
saving result
starting test year 2017
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.1326030568832282 0.894670049538585
expected return =  10.792202272609085
volatility =  0.12680818459675625
years =  0.9863013698630136
sharpe ratio =  0.5868780469103992
beat percent =  0.7243676636153875
saving result
starting test year 2018
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.4980572176854936 0.6194359211634775
expected return =  -6.15102243445

volatility =  0.7077892339285915
years =  0.9945205479452055
sharpe ratio =  2.170104819722105
beat percent =  162.94226408791639
saving result
starting test year 2003
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.5450522669949085 0.5862369375999515
expected return =  114.28615470647436
volatility =  0.557841527289556
years =  0.9945205479452055
sharpe ratio =  1.9972666974315996
beat percent =  107.38466964594606
saving result
starting test year 2004
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.20619091728933184 0.8368278623728644
expected return =  104.01619568806844
volatility =  0.5115221731496917
years =  0.9972602739726028
sharpe ratio =  1.9706272720641695
beat percent =  103.25828044091008
saving result
starting test year 2005
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance

sharpe ratio =  0.2852379455494209
beat percent =  15.818398534479716
saving result
starting test year 2011
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.32346734871180866 0.7468118385185546
expected return =  -12.844153676244186
volatility =  0.2157719751657433
years =  0.989041095890411
sharpe ratio =  -0.7640691593909636
beat percent =  -12.645795666284888
saving result
starting test year 2012
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.7870107099535928 0.076090178193609
expected return =  93.47002815718693
volatility =  0.3245510716331229
years =  0.9945205479452055
sharpe ratio =  2.788005415693213
beat percent =  89.72128156609743
saving result
starting test year 2013
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.611022413489962 0.5420109135690453
exp

sharpe ratio =  -0.45472182718917625
beat percent =  3.411560108456457
saving result
starting test year 2019
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.2996215406462703 0.19554284699821095
expected return =  19.40511740037323
volatility =  0.1330185148127032
years =  0.7890410958904109
sharpe ratio =  1.5857409909800146
beat percent =  7.418346989347736
saving result
formatting summary
Runnning ETL for UTX
formatting
preprocessing
begin test itterations
starting test year 2002
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.0621996537218747 0.9504730526696344
expected return =  -1.6533889000524438
volatility =  0.23154706081856472
years =  0.9945205479452055
sharpe ratio =  -0.2229567701061949
beat percent =  5.980507241315547
saving result
starting test year 2003
test-train split
training shapes
optimizing trade thresholds
computin

,symbol,equity,test_year,price$,lowT,highT,#trades,in_days,t-val,p-val,exp_ret%,volatility,years,sharpe,beat%
0,ABT,ABBOTT LABORATORIES,2002,11.314899,0.31250,0.79375,13.0,99,4.051459,0.000074,30.835651,0.147148,0.994521,1.869238,24.331917
1,ABT,ABBOTT LABORATORIES,2003,14.015966,0.27500,0.73750,20.0,100,-0.796166,0.426895,4.050214,0.120568,0.994521,0.047486,-0.523296
2,ABT,ABBOTT LABORATORIES,2004,15.372593,0.41250,0.60000,38.0,128,1.561969,0.119599,15.532756,0.103041,0.997260,1.171907,9.077632
3,ABT,ABBOTT LABORATORIES,2005,13.312641,0.40625,0.65625,18.0,99,1.579806,0.115950,8.191178,0.074593,0.989041,0.641070,5.821230
4,ABT,ABBOTT LABORATORIES,2006,16.874145,0.33750,0.80000,19.0,94,0.275280,0.783407,12.755089,0.082894,0.986301,1.137870,6.819800


The ticker format of "BIO.B" seems to confuse the data server, so we'll try skipping it.

In [33]:
print(hiccup2[1:])

['BX', 'BRC', 'BR', 'CCMP', 'CACI', 'CWT', 'CNI', 'CCBG', 'CTRE', 'CSL', 'CTLT', 'CBOE', 'CE', 'CF', 'GIB', 'CHEF', 'CB', 'CHD', 'CINF', 'CSCO', 'CHCO', 'CME', 'KO', 'CNS', 'CIGI', 'CBU', 'CWCO', 'CACC', 'CSX', 'CW', 'DHI', 'DHR', 'DRI', 'DE', 'DAL', 'DIOD', 'DFS', 'DTV', 'DTE', 'ETN', 'ECL', 'ESLT', 'EME', 'EBF', 'ENTG', 'EPAM', 'ESE', 'EVRG', 'EXPD', 'EXR', 'FSS', 'FNF', 'FAF', 'FCAP', 'FFIN', 'FCFS', 'FLO', 'FMC', 'WILC', 'GRMN', 'GWR', 'GNTX', 'GGG', 'THG', 'HE', 'HEI.A', 'HXL', 'HON', 'ICLR', 'IDA', 'IEX', 'ITW', 'IOR', 'INFY', 'IR', 'ICE', 'JJSF', 'KSU', 'KNSL', 'KLAC', 'LHX', 'LRCX', 'LDOS', 'LIN', 'LMT']


In [32]:
len(hiccup2)

86

In [34]:
# ticker list for etl runs
t_list = df_stocks.symbol.tolist()
e_list = df_stocks.equity.tolist()
hiccup3 = []
# run etl
for ticker, equity in zip(t_list,e_list):
    if ticker in hiccup2[1:]:
        try:
            df_results, df_shape = run_etl(ticker, equity)
            df_res = pd.concat([df_res,df_results], ignore_index=True)
        except:
            hiccup3.append(ticker)

df_res.to_csv('stock_survey_102819c.csv', index=None)
df_res.head()

Runnning ETL for BX
formatting
preprocessing
begin test itterations
starting test year 2010
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  3.0553993381550475 0.0025646737310223667
expected return =  41.94356015395988
volatility =  0.22879463780717396
years =  0.989041095890411
sharpe ratio =  1.7005778245430294
beat percent =  22.245284129872587
saving result
starting test year 2011
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.9262191438111869 0.055384403599747195
expected return =  75.92111233408481
volatility =  0.2720916307245963
years =  0.989041095890411
sharpe ratio =  2.6925614470602897
beat percent =  50.09808169059578
saving result
starting test year 2012
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.5982020754276611 0.5502646170328869
expected return 

years =  0.9972602739726028
sharpe ratio =  -0.04494986228413935
beat percent =  14.923128852105837
saving result
starting test year 2009
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.9509790150281683 0.0035819797537683415
expected return =  84.19475592869841
volatility =  0.28060089522604287
years =  0.9945205479452055
sharpe ratio =  2.8923157503148684
beat percent =  42.95643386011402
saving result
starting test year 2010
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.4821563393183153 0.13981660155814574
expected return =  40.870330334175264
volatility =  0.16382906858743018
years =  0.989041095890411
sharpe ratio =  2.3086981758550564
beat percent =  15.222991573182341
saving result
starting test year 2011
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.4603

beat percent =  53.570467240244746
saving result
starting test year 2017
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.0628581907431336 0.04089803345030118
expected return =  36.6696394353546
volatility =  0.1299720682338537
years =  0.9863013698630136
sharpe ratio =  2.5912444451119026
beat percent =  24.596200747290343
saving result
starting test year 2018
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.675017586512401 0.09685250509010225
expected return =  17.886502729590916
volatility =  0.1614760093229781
years =  0.9945205479452055
sharpe ratio =  0.8970404343486055
beat percent =  19.43363396279866
saving result
starting test year 2019
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.3296480941488997 0.18542719646771466
expected return =  26.279805380649602

expected return =  25.28785323849845
volatility =  0.09577850643450204
years =  0.9945205479452055
sharpe ratio =  2.2893633519521894
beat percent =  5.191622049214796
saving result
starting test year 2004
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.436127450943087 0.015978997830766773
expected return =  27.71204299656407
volatility =  0.06933402605547791
years =  0.9972602739726028
sharpe ratio =  3.5030671611921225
beat percent =  13.395708452383914
saving result
starting test year 2005
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.7065079282986384 0.0896278618799463
expected return =  9.527087042760218
volatility =  0.13356546361438984
years =  0.989041095890411
sharpe ratio =  0.45914940645120716
beat percent =  5.405725679257434
saving result
starting test year 2006
test-train split
training shapes
optimizing trade thresholds
c

optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.9563186143525281 0.3402982719061375
expected return =  33.18896298255438
volatility =  0.17039540694570313
years =  0.9945205479452055
sharpe ratio =  1.7530884674756793
beat percent =  16.431882062570043
saving result
starting test year 2014
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.4066456245700762 0.01716206140671647
expected return =  49.465092264472
volatility =  0.15688713934048182
years =  0.9945205479452055
sharpe ratio =  2.9471904072220623
beat percent =  44.98183489715797
saving result
starting test year 2015
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.778510733693271 0.4370071233758024
expected return =  13.681567013281715
volatility =  0.2122859365497473
years =  0.9945205479452055
sharpe ratio =  0.48316660606420014
beat perce

...

years =  0.989041095890411
sharpe ratio =  -0.9592405058408106
beat percent =  -14.990992315836326
saving result
starting test year 2011
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.9908108245395383 0.3233597210048511
expected return =  27.083876132767813
volatility =  0.13671994088691306
years =  0.989041095890411
sharpe ratio =  1.7469269170489812
beat percent =  24.997342151206034
saving result
starting test year 2012
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.8140911443085765 0.41679813251833253
expected return =  15.50177549618419
volatility =  0.1432410711937186
years =  0.9945205479452055
sharpe ratio =  0.8438351254174488
beat percent =  2.5732681665430643
saving result
starting test year 2013
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.8522927

beat percent =  3.838836399525258
saving result
starting test year 2019
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.564474240871537 0.5731976617787875
expected return =  17.886174577011626
volatility =  0.20522705809267444
years =  0.7917808219178082
sharpe ratio =  0.9301797452557787
beat percent =  8.426009919239718
saving result
formatting summary
Runnning ETL for GRMN
formatting
preprocessing
begin test itterations
starting test year 2004
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  3.836905084038804 0.00016816891779313702
expected return =  71.25855895059887
volatility =  0.23457562115863492
years =  0.9972602739726028
sharpe ratio =  2.8969047971309068
beat percent =  63.12241017627784
saving result
starting test year 2005
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-va

sharpe ratio =  3.2853118812725546
beat percent =  73.21921630226521
saving result
starting test year 2011
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.1291196695361236 0.0358871812578279
expected return =  34.17092276426366
volatility =  0.2978154685441341
years =  0.989041095890411
sharpe ratio =  1.042576737166554
beat percent =  21.179766445020753
saving result
starting test year 2012
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.8608023166826485 0.3910675436809835
expected return =  -17.17273042485842
volatility =  0.38679562434400916
years =  0.9945205479452055
sharpe ratio =  -0.5369074703134601
beat percent =  -30.46953064624579
saving result
starting test year 2013
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.5296773895755005 0.1285066484372192
exp

beat percent =  11.981094903036116
saving result
starting test year 2019
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.43395726282922636 0.6654075527967878
expected return =  16.81552041203169
volatility =  0.09408325663736125
years =  0.7917808219178082
sharpe ratio =  1.885308355388085
beat percent =  -0.2866109894810194
saving result
formatting summary
Runnning ETL for HE
formatting
preprocessing
begin test itterations
starting test year 2002
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.5124422528359864 0.012957224383716052
expected return =  37.41588535262754
volatility =  0.13735618836074961
years =  0.9945205479452055
sharpe ratio =  2.4842006675137718
beat percent =  49.677249677083
saving result
starting test year 2003
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value

,symbol,equity,test_year,price$,lowT,highT,#trades,in_days,t-val,p-val,exp_ret%,volatility,years,sharpe,beat%
0,ABT,ABBOTT LABORATORIES,2002,11.314899,0.31250,0.79375,13.0,99,4.051459,0.000074,30.835651,0.147148,0.994521,1.869238,24.331917
1,ABT,ABBOTT LABORATORIES,2003,14.015966,0.27500,0.73750,20.0,100,-0.796166,0.426895,4.050214,0.120568,0.994521,0.047486,-0.523296
2,ABT,ABBOTT LABORATORIES,2004,15.372593,0.41250,0.60000,38.0,128,1.561969,0.119599,15.532756,0.103041,0.997260,1.171907,9.077632
3,ABT,ABBOTT LABORATORIES,2005,13.312641,0.40625,0.65625,18.0,99,1.579806,0.115950,8.191178,0.074593,0.989041,0.641070,5.821230
4,ABT,ABBOTT LABORATORIES,2006,16.874145,0.33750,0.80000,19.0,94,0.275280,0.783407,12.755089,0.082894,0.986301,1.137870,6.819800


Similarly, the format of "HEI.A" seems to confuse the data server, so we'll try skipping that also.

In [37]:
print(hiccup3[1:])

['HXL', 'HON', 'ICLR', 'IDA', 'IEX', 'ITW', 'IOR', 'INFY', 'IR', 'ICE', 'JJSF', 'KSU', 'KNSL', 'KLAC', 'LHX', 'LRCX', 'LDOS', 'LIN', 'LMT']


In [36]:
len(hiccup3)

20

In [38]:
# ticker list for etl runs
t_list = df_stocks.symbol.tolist()
e_list = df_stocks.equity.tolist()
hiccup4 = []
# run etl
for ticker, equity in zip(t_list,e_list):
    if ticker in hiccup3[1:]:
        try:
            df_results, df_shape = run_etl(ticker, equity)
            df_res = pd.concat([df_res,df_results], ignore_index=True)
        except:
            hiccup4.append(ticker)

df_res.to_csv('stock_survey_102819d.csv', index=None)
df_res.head()

Runnning ETL for HXL
formatting
preprocessing
begin test itterations
starting test year 2002
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.7740563169207043 0.4397906537286812
expected return =  37.81270199369358
volatility =  0.4831631785224012
years =  0.9945205479452055
sharpe ratio =  0.7144798687994247
beat percent =  47.701408208351076
saving result
starting test year 2003
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -1.2680421482294502 0.21163852243695952
expected return =  90.74067964162073
volatility =  0.4754822990544652
years =  0.9945205479452055
sharpe ratio =  1.8452974687861825
beat percent =  66.30130831605764
saving result
starting test year 2004
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -1.2239288937401596 0.22265214692477264
expected return

beat percent =  44.77606247742993
saving result
starting test year 2010
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.9932460192806584 0.3216248822556511
expected return =  24.067865886989637
volatility =  0.21502321746786301
years =  0.989041095890411
sharpe ratio =  0.9689440092454206
beat percent =  6.463205950695141
saving result
starting test year 2011
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.031982610286188234 0.9745226283451028
expected return =  -12.797068333109053
volatility =  0.23181829111501298
years =  0.989041095890411
sharpe ratio =  -0.7091271368210176
beat percent =  -22.656051811319536
saving result
starting test year 2012
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.41113230823927127 0.6813625691570314
expected return =  42.347284039

saving result
starting test year 2018
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.738152495815847 0.006705032531553096
expected return =  24.865471660699654
volatility =  0.13306582754075813
years =  0.9945205479452055
sharpe ratio =  1.615927387365779
beat percent =  24.33511321323185
saving result
starting test year 2019
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.07167236369872614 0.9429966592209228
expected return =  26.437228500089425
volatility =  0.1554065897731605
years =  0.7917808219178082
sharpe ratio =  1.9233147893629332
beat percent =  11.402350540292769
saving result
formatting summary
Runnning ETL for ITW
formatting
preprocessing
begin test itterations
starting test year 2002
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.9962271208198645 0

beat percent =  0.6945665185996042
saving result
starting test year 2008
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.3506322140331022 0.17828117436959134
expected return =  -3.2038780966977654
volatility =  0.4054702261430956
years =  0.9972602739726028
sharpe ratio =  -0.16555296854846727
beat percent =  -14.357577961407342
saving result
starting test year 2009
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -1.0107454969422085 0.31316208737879037
expected return =  22.609345579667718
volatility =  0.24950640637575267
years =  0.9945205479452055
sharpe ratio =  0.770878602124344
beat percent =  18.635248769230795
saving result
starting test year 2010
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.040740367072477725 0.9675479574487753
expected return =  30.23563

sharpe ratio =  -0.4785779164862994
beat percent =  -11.492586084945579
saving result
starting test year 2004
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  -0.7679549303486924 0.44333751530666843
expected return =  -1.3073306579634014
volatility =  0.1573032298331613
years =  0.9972602739726028
sharpe ratio =  -0.30583747268302197
beat percent =  -1.554540719377484
saving result
starting test year 2005
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.3842743073651167 0.16757138668408397
expected return =  34.40585962373008
volatility =  0.16392196644976303
years =  0.989041095890411
sharpe ratio =  1.908657433950463
beat percent =  29.55035174629692
saving result
starting test year 2006
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.1989075856043816 0.0290139861469

volatility =  0.1162197844577445
years =  0.989041095890411
sharpe ratio =  0.9427989742434434
beat percent =  9.036182865708442
saving result
starting test year 2011
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.371861329413527 0.17164492239843573
expected return =  42.307077054519524
volatility =  0.2115844036562793
years =  0.989041095890411
sharpe ratio =  1.856273565200102
beat percent =  23.113798629665204
saving result
starting test year 2012
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.1293738071799737 0.26025042520557506
expected return =  17.460327514508457
volatility =  0.16302010039856588
years =  0.9945205479452055
sharpe ratio =  0.8622573309510244
beat percent =  1.723482214934947
saving result
starting test year 2013
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t

sharpe ratio =  -1.0724589883402456
beat percent =  -22.54545231290568
saving result
starting test year 2019
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.042072395973135 0.04260603798735346
expected return =  83.05450299963668
volatility =  0.21091809567417694
years =  0.7917808219178082
sharpe ratio =  4.807355445898353
beat percent =  64.99636394402458
saving result
formatting summary
Runnning ETL for LDOS
formatting
preprocessing
begin test itterations
starting test year 2009
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.819758445422848 0.41338160494572695
expected return =  5.837419609683314
volatility =  0.12898974715646186
years =  0.9945205479452055
sharpe ratio =  0.18370310458260436
beat percent =  -2.8190173639746963
saving result
starting test year 2010
test-train split
training shapes
optimizing trade thresholds
computing

volatility =  0.1297283195616062
years =  0.9945205479452055
sharpe ratio =  0.6582667754602489
beat percent =  14.148677825962054
saving result
starting test year 2016
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.062303774142255 0.04057186033245254
expected return =  36.086847831892484
volatility =  0.12256345233820226
years =  0.989041095890411
sharpe ratio =  2.6913979226428983
beat percent =  34.86851175333343
saving result
starting test year 2017
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.332560338248462 0.18427250415111657
expected return =  27.018551679734614
volatility =  0.07269974264592434
years =  0.9863013698630136
sharpe ratio =  3.286642905783749
beat percent =  17.39211368675737
saving result
starting test year 2018
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance


,symbol,equity,test_year,price$,lowT,highT,#trades,in_days,t-val,p-val,exp_ret%,volatility,years,sharpe,beat%
0,ABT,ABBOTT LABORATORIES,2002,11.314899,0.31250,0.79375,13.0,99,4.051459,0.000074,30.835651,0.147148,0.994521,1.869238,24.331917
1,ABT,ABBOTT LABORATORIES,2003,14.015966,0.27500,0.73750,20.0,100,-0.796166,0.426895,4.050214,0.120568,0.994521,0.047486,-0.523296
2,ABT,ABBOTT LABORATORIES,2004,15.372593,0.41250,0.60000,38.0,128,1.561969,0.119599,15.532756,0.103041,0.997260,1.171907,9.077632
3,ABT,ABBOTT LABORATORIES,2005,13.312641,0.40625,0.65625,18.0,99,1.579806,0.115950,8.191178,0.074593,0.989041,0.641070,5.821230
4,ABT,ABBOTT LABORATORIES,2006,16.874145,0.33750,0.80000,19.0,94,0.275280,0.783407,12.755089,0.082894,0.986301,1.137870,6.819800


In [39]:
print(hiccup4)

[]


In [40]:
# ticker list for etl runs
t_list = ['BIO-B','HEI-A']
e_list = ['BIO-RAD LABORATORIES INC','HEICO CORP']
hiccup5 = []
# run etl
for ticker, equity in zip(t_list,e_list):
    try:
        df_results, df_shape = run_etl(ticker, equity)
        df_res = pd.concat([df_res,df_results], ignore_index=True)
    except:
        hiccup5.append(ticker)

df_res.to_csv('stock_survey_102819e.csv', index=None)
df_res.head()

Runnning ETL for BIO-B
formatting
preprocessing
begin test itterations
starting test year 2002
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  0.05487856312956253 0.9563134523136843
expected return =  37.216468745812634
volatility =  0.35638110713730314
years =  0.9945205479452055
sharpe ratio =  0.9518326724127145
beat percent =  55.31234275501508
saving result
starting test year 2003
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  2.2914141761884776 0.024946250536543025
expected return =  80.85266550153361
volatility =  0.39514743686220094
years =  0.9945205479452055
sharpe ratio =  1.9688381468630762
beat percent =  58.84974243742669
saving result
starting test year 2004
test-train split
training shapes
optimizing trade thresholds
computing trades
evaluating performance
t-value, p-value =  1.6228035862912193 0.10653537243234815
expected re

,symbol,equity,test_year,price$,lowT,highT,#trades,in_days,t-val,p-val,exp_ret%,volatility,years,sharpe,beat%
0,ABT,ABBOTT LABORATORIES,2002,11.314899,0.31250,0.79375,13.0,99,4.051459,0.000074,30.835651,0.147148,0.994521,1.869238,24.331917
1,ABT,ABBOTT LABORATORIES,2003,14.015966,0.27500,0.73750,20.0,100,-0.796166,0.426895,4.050214,0.120568,0.994521,0.047486,-0.523296
2,ABT,ABBOTT LABORATORIES,2004,15.372593,0.41250,0.60000,38.0,128,1.561969,0.119599,15.532756,0.103041,0.997260,1.171907,9.077632
3,ABT,ABBOTT LABORATORIES,2005,13.312641,0.40625,0.65625,18.0,99,1.579806,0.115950,8.191178,0.074593,0.989041,0.641070,5.821230
4,ABT,ABBOTT LABORATORIES,2006,16.874145,0.33750,0.80000,19.0,94,0.275280,0.783407,12.755089,0.082894,0.986301,1.137870,6.819800


In [41]:
print(hiccup5)

[]


### Disclaimer: this notebook is intended for educational purposes only and not recommended for real trading.